# T81-558: Applications of Deep Neural Networks
**Module 7: Generative Adversarial Networks**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module Video Material

Main video lecture:

* [Part 7.1: Introduction to GANS for Image and Data Generation](https://www.youtube.com/watch?v=u8xn393mDPM&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN&index=21)
* [Part 7.2: Implementing a GAN in Keras](https://www.youtube.com/watch?v=cf6FDLFNWEk&index=22&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN)
* [Part 7.3: Face Generation with StyleGAN and Python](https://www.youtube.com/watch?v=LSSH_NdXwhU&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN&index=23)
* [Part 7.4: GANS for Semi-Supervised Learning in Keras](https://www.youtube.com/watch?v=LSSH_NdXwhU&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN&index=23)
* [Part 7.5: An Overview of GAN Research](https://www.youtube.com/watch?v=LSSH_NdXwhU&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN&index=23)


# Part 7.1: Introduction to GANS for Image and Data Generation

A generative adversarial network (GAN) is a class of machine learning systems invented by Ian Goodfellow in 2014. Two neural networks contest with each other in a game. Given a training set, this technique learns to generate new data with the same statistics as the training set. For example, a GAN trained on photographs can generate new photographs that look at least superficially authentic to human observers, having many realistic characteristics. Though originally proposed as a form of generative model for unsupervised learning, GANs have also proven useful for semi-supervised learning, fully supervised learning, and reinforcement learning.  GANs were introduced in the following paper: 

* Goodfellow, I., Pouget-Abadie, J., Mirza, M., Xu, B., Warde-Farley, D., Ozair, S., ... & Bengio, Y. (2014). [Generative adversarial nets](https://papers.nips.cc/paper/5423-generative-adversarial-nets.pdf). In *Advances in neural information processing systems* (pp. 2672-2680).

This paper used neural networks to automatically generate images for several datasets that we've seen previously: MINST and CIFAR.  However, it also included the Toronto Face Dataset (a private dataset used by some researchers).

![GAN](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/gan-2.png "GAN")

Only sub-figure D made use of convolutional neural networks.  Figures A-C make use of fully connected neural networks.  As we will see in this module, the role of convolutional neural networks with GANs was greatly increased.

A GAN is called a generative model because it generates new data.  The overall process of a GAN is given by the following diagram.

![GAN](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/gan-1.png "GAN")

# Part 7.2: Implementing DCGANs in Keras

Paper that described the type of DCGAN that we will create in this module.

* Radford, A., Metz, L., & Chintala, S. (2015). [Unsupervised representation learning with deep convolutional generative adversarial networks](https://arxiv.org/abs/1511.06434). *arXiv preprint arXiv:1511.06434*.

This paper implements a DCGAN as follows:

* No pre-processing was applied to training images besides scaling to the range of the tanh activation function [-1, 1]. 
* All models were trained with mini-batch stochastic gradient descent (SGD) with a mini-batch size of 128. 
* All weights were initialized from a zero-centered Normal distribution with standard deviation 0.02. 
* In the LeakyReLU, the slope of the leak was set to 0.2 in all models.
* we used the Adam optimizer(Kingma & Ba, 2014) with tuned hyperparameters. We found the suggested learning rate of 0.001, to be too high, using 0.0002 instead. 
* Additionally, we found leaving the momentum term $\beta{1}$ at the suggested value of 0.9 resulted in training oscillation and instability while reducing it to 0.5 helped stabilize training.

The paper also provides the following architecture guidelines for stable Deep Convolutional GANs:

* Replace any pooling layers with strided convolutions (discriminator) and fractional-strided convolutions (generator).
* Use batchnorm in both the generator and the discriminator.
* Remove fully connected hidden layers for deeper architectures.
* Use ReLU activation in generator for all layers except for the output, which uses Tanh.
* Use LeakyReLU activation in the discriminator for all layers.

While creating the material for this module I used a number of Internet resources, some of the most helpful were:

* [Keep Calm and train a GAN. Pitfalls and Tips on training Generative Adversarial Networks](https://medium.com/@utk.is.here/keep-calm-and-train-a-gan-pitfalls-and-tips-on-training-generative-adversarial-networks-edd529764aa9)
* [Collection of Keras implementations of Generative Adversarial Networks GANs](https://github.com/eriklindernoren/Keras-GAN)
* [dcgan-facegenerator](https://github.com/platonovsimeon/dcgan-facegenerator), [Semi-Paywalled Article by GitHub Author](https://medium.com/datadriveninvestor/generating-human-faces-with-keras-3ccd54c17f16)

The program created next will generate faces similar to these.  While these faces are not perfect, they demonstrate how we can construct and train a GAN on or own.  Later we will see how to import very advanced weights from nVidia to produce high resolution, realistic looking faces.

![GAN](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/gan-3.png "GAN")

The following packages will be used to implement a basic GAN system in Python/Keras.

In [ ]:
from keras.layers import Input, Reshape, Dropout, Dense, Flatten, BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model, load_model
from keras.optimizers import Adam
import numpy as np
from PIL import Image
from tqdm import tqdm
import os 

I suggest running this code with a GPU, it will be very slow on a CPU alone.  The following code mounts your Google drive for use with Google CoLab.  If you are not using CoLab, the following code will not work.

In [ ]:
# Run this for Google CoLab
from google.colab import drive
drive.mount('/content/drive')

These are the constants that define how the GANs will be created for this example.  The higher the resolution, the more memory that will be needed.  Higher resolution will also result in longer run times.  For Google CoLab (with GPU) 128x128 resolution is as high as can be used (due to memoru).  Note that the resolution is specified as a multiple of 32.  So **GENERATE_RES** of 1 is 32, 2 is 64, etc.

To run this you will need training data.  The training data can be any collection of images.  I suggest using training data from the following two locations.  Simply unzip and combine to a common directory.  This directory should be uploaded to Google Drive (if you are using CoLab). The constant **DATA_PATH** defines where these images are stored.

The source data (faces) used in this module can be found here:

* [Kaggle Faces Data New](https://www.kaggle.com/gasgallo/faces-data-new)
* [Kaggle Lag Dataset: Dataset of faces, from more than 1k different subjects](https://www.kaggle.com/gasgallo/lag-dataset)

In [ ]:
# Generation resolution - Must be square 
# Training data is also scaled to this.
# Note GENERATE_RES higher than 4 will blow Google CoLab's memory.
GENERATE_RES = 2 # (1=32, 2=64, 3=96, etc.)
GENERATE_SQUARE = 32 * GENERATE_RES # rows/cols (should be square)
IMAGE_CHANNELS = 3

# Preview image 
PREVIEW_ROWS = 4
PREVIEW_COLS = 7
PREVIEW_MARGIN = 16
SAVE_FREQ = 100

# Size vector to generate images from
SEED_SIZE = 100

# Configuration
DATA_PATH = '/content/drive/My Drive/projects/faces'
EPOCHS = 10000
BATCH_SIZE = 32

print(f"Will generate {GENERATE_SQUARE}px square images.")

Next we will load and preprocess the images.  This can take awhile.  Google CoLab took around an hour to process.  Because of this we store the processed file as a binary.  This way we can simply reload the processed training data and quickly use it.  It is most efficient to only perform this operation once.  The dimensions of the image are encoded into the filename of the binary file because we need to regenerate it if these change.

In [ ]:
# Image set has 11,682 images.  Can take over an hour for initial preprocessing.
# Because of this time needed, save a Numpy preprocessed file.
# Note, that file is large enough to cause problems for sume verisons of Pickle,
# so Numpy binary files are used.
training_binary_path = os.path.join(DATA_PATH,f'training_data_{GENERATE_SQUARE}_{GENERATE_SQUARE}.npy')

print(f"Looking for file: {training_binary_path}")

if not os.path.isfile(training_binary_path):
  print("Loading training images...")

  training_data = []
  faces_path = os.path.join(DATA_PATH,'face_images')
  for filename in tqdm(os.listdir(faces_path)):
      path = os.path.join(faces_path,filename)
      image = Image.open(path).resize((GENERATE_SQUARE,GENERATE_SQUARE),Image.ANTIALIAS)
      training_data.append(np.asarray(image))
  training_data = np.reshape(training_data,(-1,GENERATE_SQUARE,GENERATE_SQUARE,IMAGE_CHANNELS))
  training_data = training_data / 127.5 - 1.

  print("Saving training image binary...")
  np.save(training_binary_path,training_data)
else:
  print("Loading previous training pickle...")
  training_data = np.load(training_binary_path)

The code below creates the generator and discriminator.

In [ ]:
def build_generator(seed_size, channels):
    model = Sequential()

    model.add(Dense(4*4*256,activation="relu",input_dim=seed_size))
    model.add(Reshape((4,4,256)))

    model.add(UpSampling2D())
    model.add(Conv2D(256,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))

    model.add(UpSampling2D())
    model.add(Conv2D(256,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
   
    # Output resolution, additional upsampling
    for i in range(GENERATE_RES):
      model.add(UpSampling2D())
      model.add(Conv2D(128,kernel_size=3,padding="same"))
      model.add(BatchNormalization(momentum=0.8))
      model.add(Activation("relu"))

    # Final CNN layer
    model.add(Conv2D(channels,kernel_size=3,padding="same"))
    model.add(Activation("tanh"))

    input = Input(shape=(seed_size,))
    generated_image = model(input)

    return Model(input,generated_image)


def build_discriminator(image_shape):
    model = Sequential()

    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=image_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(512, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    input_image = Input(shape=image_shape)

    validity = model(input_image)

    return Model(input_image, validity)
  
def save_images(cnt,noise):
  image_array = np.full(( 
      PREVIEW_MARGIN + (PREVIEW_ROWS * (GENERATE_SQUARE+PREVIEW_MARGIN)), 
      PREVIEW_MARGIN + (PREVIEW_COLS * (GENERATE_SQUARE+PREVIEW_MARGIN)), 3), 
      255, dtype=np.uint8)
  
  generated_images = generator.predict(noise)

  generated_images = 0.5 * generated_images + 0.5

  image_count = 0
  for row in range(PREVIEW_ROWS):
      for col in range(PREVIEW_COLS):
        r = row * (GENERATE_SQUARE+16) + PREVIEW_MARGIN
        c = col * (GENERATE_SQUARE+16) + PREVIEW_MARGIN
        image_array[r:r+GENERATE_SQUARE,c:c+GENERATE_SQUARE] = generated_images[image_count] * 255
        image_count += 1

          
  output_path = os.path.join(DATA_PATH,'output')
  if not os.path.exists(output_path):
    os.makedirs(output_path)
  
  filename = os.path.join(output_path,f"train-{cnt}.png")
  im = Image.fromarray(image_array)
  im.save(filename)

Next we actually build the discriminator and the generator.  Both will be trained with the Adam optimizer.

In [ ]:
image_shape = (GENERATE_SQUARE,GENERATE_SQUARE,IMAGE_CHANNELS)
optimizer = Adam(1.5e-4,0.5) # learning rate and momentum adjusted from paper

discriminator = build_discriminator(image_shape)
discriminator.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=["accuracy"])
generator = build_generator(SEED_SIZE,IMAGE_CHANNELS)

random_input = Input(shape=(SEED_SIZE,))

generated_image = generator(random_input)

discriminator.trainable = False

validity = discriminator(generated_image)

combined = Model(random_input,validity)
combined.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=["accuracy"])


In [ ]:
y_real = np.ones((BATCH_SIZE,1))
y_fake = np.zeros((BATCH_SIZE,1))

fixed_seed = np.random.normal(0, 1, (PREVIEW_ROWS * PREVIEW_COLS, SEED_SIZE))

cnt = 1
for epoch in range(EPOCHS):
    idx = np.random.randint(0,training_data.shape[0],BATCH_SIZE)
    x_real = training_data[idx]

    # Generate some images
    seed = np.random.normal(0,1,(BATCH_SIZE,SEED_SIZE))
    x_fake = generator.predict(seed)

    # Train discriminator on real and fake
    discriminator_metric_real = discriminator.train_on_batch(x_real,y_real)
    discriminator_metric_generated = discriminator.train_on_batch(x_fake,y_fake)
    discriminator_metric = 0.5 * np.add(discriminator_metric_real,discriminator_metric_generated)
    
    # Train generator on Calculate losses
    generator_metric = combined.train_on_batch(seed,y_real)
    
    # Time for an update?
    if epoch % SAVE_FREQ == 0:
        save_images(cnt, fixed_seed)
        cnt += 1
        print(f"Epoch {epoch}, Discriminator accuarcy: {discriminator_metric[1]}, Generator accuracy: {generator_metric[1]}")
        
generator.save(os.path.join(DATA_PATH,"face_generator.h5"))

# Part 7.3: Face Generation with StyleGAN and Python

GANs have appeared frequently in the media, showcasing their ability to generate extremely photorealistic faces.  One significant step forward for realistic face generation was nVidia StyleGAN, which was introduced in the following paper.

* Karras, T., Laine, S., & Aila, T. (2018). [A style-based generator architecture for generative adversarial networks](https://arxiv.org/abs/1812.04948). *arXiv preprint arXiv:1812.04948*.

In this part we will make use of StyleGAN.  We will also preload weights that nVidia trained on.  This will allow us to generate high resolution photorealistic looking faces, such as this one.

![GAN](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/gan-example246.png "GAN")

The above image was generated with StyleGAN, using Google CoLab.  Following the instructions in this section, you will be able to create faces like this of your own.  

While the above image looks much more realistic than the previous set of images, it is not perfect.  There are usually a number of tell-tail signs that you are looking at a computer generated image.  One of the most obvious is usually the surreal, dream-like backgrounds.  The background does not look obviously fake, at first glance; however, upon closer inspection you usually can't quite discern exactly what a GAN generated background actually is.  Also look at the image character's left eye.  It is slightly unrealistic looking, especially near the eyelashes.

Look at the following GAN face.  Can you spot any imperfections?

![GAN](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/gan-example221.png "GAN")

Notice the earrings?  GANs sometimes have problems with symmetry, particularly earrings.

### Keras Sequence vs Functional Model API

Most of the neural networks create in this course have made use of the Keras sequence object.  You might have noticed that we briefly made use of another type of neural network object for the ResNet, the Model.  These are the [two major means](https://keras.io/getting-started/functional-api-guide/) of constructing a neural network in Keras:

* [Sequential](https://keras.io/getting-started/sequential-model-guide/) - Simplified interface to Keras that supports most models where the flow of information is a simple sequence from input to output. 
* [Keras Functional API](https://keras.io/getting-started/functional-api-guide/) - More complex interface that allows neural networks to be constructed of reused layers, multiple input layers, and supports building your own recurrent connections.

It is important to point out that these are not two specific types of neural network.  Rather, they are two means of constructing neural networks in Keras.  Some types of neural network can be implemented in either, such as dense feedforward neural networks (like we used for the Iris and MPG datasets).  However, other types of neural network, like ResNet and GANs can only be used in the Functional Model API.

### Generating High Rez GAN Faces with Google CoLab

This notebook demonstrates how to run [NVidia StyleGAN](https://github.com/NVlabs/stylegan) inside of a Google CoLab notebook.  I suggest you use this to generate GAN faces from a pretrained model.  If you try to train your own, you will run into compute limitations of Google CoLab.

Make sure to run this code on a GPU instance.  GPU is assumed.

First, map your G-Drive, this is where your GANs will be written to.

In [ ]:
# Run this for Google CoLab
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Next, clone StyleGAN from GitHub.

In [ ]:
!git clone https://github.com/NVlabs/stylegan.git

Verify that StyleGAN has been cloned.

In [ ]:
!ls /content/stylegan/

Add the StyleGAN folder to Python so that you can import it.

In [ ]:
import sys
sys.path.insert(0, "/content/stylegan")

import dnnlib

The code below is based on code from NVidia. This actually generates your images.

In [ ]:
# Copyright (c) 2019, NVIDIA CORPORATION. All rights reserved.
#
# This work is licensed under the Creative Commons Attribution-NonCommercial
# 4.0 International License. To view a copy of this license, visit
# http://creativecommons.org/licenses/by-nc/4.0/ or send a letter to
# Creative Commons, PO Box 1866, Mountain View, CA 94042, USA.

"""Minimal script for generating an image using pre-trained StyleGAN generator."""

import os
import pickle
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import config

def main():
    # Initialize TensorFlow.
    tflib.init_tf()

    # Load pre-trained network.
    url = 'https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ' # karras2019stylegan-ffhq-1024x1024.pkl
    with dnnlib.util.open_url(url, cache_dir=config.cache_dir) as f:
        _G, _D, Gs = pickle.load(f)
        # _G = Instantaneous snapshot of the generator. Mainly useful for resuming a previous training run.
        # _D = Instantaneous snapshot of the discriminator. Mainly useful for resuming a previous training run.
        # Gs = Long-term average of the generator. Yields higher-quality results than the instantaneous snapshot.

    # Print network details.
    Gs.print_layers()

    # Pick latent vector.
    rnd = np.random.RandomState()
    

    latents = rnd.randn(1, Gs.input_shape[1])

    # Generate image.
    fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    images = Gs.run(latents, None, truncation_psi=0.7, randomize_noise=True, output_transform=fmt)

    # Save image.
    os.makedirs(config.result_dir, exist_ok=True)
    png_filename = os.path.join(config.result_dir, f'/content/drive/My Drive/images/example1.png')
    PIL.Image.fromarray(images[0], 'RGB').save(png_filename)

if __name__ == "__main__":
    main()

# Part 7.4: GANS for Semi-Supervised Learning in Keras

* [Odena, A. (2016). Semi-supervised learning with generative adversarial networks. *arXiv preprint* arXiv:1606.01583.](https://arxiv.org/abs/1606.01583)

In [12]:
import pandas as pd

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv", 
    na_values=['NA', '?'])

In [13]:
df[0:10]

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino
5,15.0,8,429.0,198.0,4341,10.0,70,1,ford galaxie 500
6,14.0,8,454.0,220.0,4354,9.0,70,1,chevrolet impala
7,14.0,8,440.0,215.0,4312,8.5,70,1,plymouth fury iii
8,14.0,8,455.0,225.0,4425,10.0,70,1,pontiac catalina
9,15.0,8,390.0,190.0,3850,8.5,70,1,amc ambassador dpl


In [14]:
x = df[['cylinders','displacement','horsepower','weight','acceleration','year']].values
x

array([[   8. ,  307. ,  130. , 3504. ,   12. ,   70. ],
       [   8. ,  350. ,  165. , 3693. ,   11.5,   70. ],
       [   8. ,  318. ,  150. , 3436. ,   11. ,   70. ],
       ...,
       [   4. ,  135. ,   84. , 2295. ,   11.6,   82. ],
       [   4. ,  120. ,   79. , 2625. ,   18.6,   82. ],
       [   4. ,  119. ,   82. , 2720. ,   19.4,   82. ]])

In [15]:
from tensorflow.keras.layers import Input, Reshape, Dropout, Dense, Flatten, BatchNormalization, Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam
import numpy as np
import os 

In [16]:
# Size vector to generate autos from
SEED_SIZE = 100
BATCH_SIZE = 32
EPOCHS = 1000
SAVE_FREQ = 1
DATA_PATH = "."

In [17]:
def build_generator(seed_size,auto_size):
    model = Sequential()

    model.add(Dense(4*4*256,activation="relu",input_dim=seed_size))
    
    model.add(Dense(25, activation='relu')) # Hidden 1
    model.add(Dense(11, activation='relu')) # Hidden 2
    model.add(Dense(auto_size)) # Hidden 2

    input = Input(shape=(seed_size,))
    generated_auto = model(input)

    return Model(input,generated_auto)


def build_discriminator(image_shape):
    model = Sequential()
    
    model.add(Dense(50, input_dim=image_shape, activation='relu')) # Hidden 1
    model.add(Dense(25, activation='relu')) # Hidden 2
    model.add(Dense(1, activation='sigmoid'))

    input_image = Input(shape= (image_shape,) )

    validity = model(input_image)

    return Model(input_image, validity)

In [18]:
x.shape[1]

6

In [19]:
optimizer = Adam(1.5e-2,0.8)

discriminator = build_discriminator(x.shape[1])
discriminator.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=["accuracy"])
generator = build_generator(SEED_SIZE,x.shape[1])

random_input = Input(shape=(SEED_SIZE,))

generated_image = generator(random_input)

discriminator.trainable = False

validity = discriminator(generated_image)
discriminator.summary()

combined = Model(random_input,validity)
combined.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=["accuracy"])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 6)                 0         
_________________________________________________________________
sequential_4 (Sequential)    (None, 1)                 1651      
Total params: 3,302
Trainable params: 1,651
Non-trainable params: 1,651
_________________________________________________________________


In [20]:
y_real = np.ones((BATCH_SIZE,1))
y_fake = np.zeros((BATCH_SIZE,1))


cnt = 1
for epoch in range(EPOCHS):
    idx = np.random.randint(0,x.shape[0],BATCH_SIZE)
    x_real = x[idx]

    # Generate some images
    seed = np.random.normal(0,1,(BATCH_SIZE,SEED_SIZE))
    x_fake = generator.predict(seed)

    # Train discriminator on real and fake
    discriminator_metric_real = discriminator.train_on_batch(x_real,y_real)
    discriminator_metric_generated = discriminator.train_on_batch(x_fake,y_fake)
    discriminator_metric = 0.5 * np.add(discriminator_metric_real,discriminator_metric_generated)
    
    # Train generator on Calculate losses
    generator_metric = combined.train_on_batch(seed,y_real)
    
    # Time for an update?
    if epoch % SAVE_FREQ == 0:
        #save_images(cnt, fixed_seed)
        cnt += 1
        print(f"Epoch {epoch}, Discriminator accuarcy: {discriminator_metric[1]}, Generator accuracy: {generator_metric[1]}")
        
generator.save(os.path.join(DATA_PATH,"automobile_generator.h5"))

Epoch 0, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 1, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 2, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 3, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 4, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 5, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 6, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 7, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 8, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 9, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 10, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 11, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 12, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 13, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 14, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 15, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Ep

Epoch 21, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 22, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 23, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 24, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 25, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 26, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 27, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 28, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 29, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 30, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 31, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 32, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 33, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 34, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 35, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 36, Discriminator accuarcy: 0.5, Generator accura

Epoch 42, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 43, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 44, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 45, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 46, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 47, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 48, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 49, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 50, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 51, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 52, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 53, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 54, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 55, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 56, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 57, Discriminator accuarcy: 0.5, Generator accura

Epoch 64, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 65, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 66, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 67, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 68, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 69, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 70, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 71, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 72, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 73, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 74, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 75, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 76, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 77, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 78, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 79, Discriminator accuarcy: 0.5, Generator accura

Epoch 85, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 86, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 87, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 88, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 89, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 90, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 91, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 92, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 93, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 94, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 95, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 96, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 97, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 98, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 99, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 100, Discriminator accuarcy: 0.5, Generator accur

...

Epoch 902, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 903, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 904, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 905, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 906, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 907, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 908, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 909, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 910, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 911, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 912, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 913, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 914, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 915, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 916, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 917, Discriminator accuarcy: 0.5, 

Epoch 924, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 925, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 926, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 927, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 928, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 929, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 930, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 931, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 932, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 933, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 934, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 935, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 936, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 937, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 938, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 939, Discriminator accuarcy: 0.5, 

Epoch 945, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 946, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 947, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 948, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 949, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 950, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 951, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 952, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 953, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 954, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 955, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 956, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 957, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 958, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 959, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 960, Discriminator accuarcy: 0.5, 

Epoch 967, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 968, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 969, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 970, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 971, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 972, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 973, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 974, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 975, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 976, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 977, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 978, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 979, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 980, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 981, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 982, Discriminator accuarcy: 0.5, 

Epoch 988, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 989, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 990, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 991, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 992, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 993, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 994, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 995, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 996, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 997, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 998, Discriminator accuarcy: 0.5, Generator accuracy: 0.0
Epoch 999, Discriminator accuarcy: 0.5, Generator accuracy: 0.0


# Part 7.5: An Overview of GAN Research

* [Keras Implementations of Generative Adversarial Networks](https://github.com/eriklindernoren/Keras-GAN)
* [Curated List of Awesome GAN Applications and Demo](https://github.com/nashory/gans-awesome-applications)

# Module 7 Assignment

You can find the first assignment here: [assignment 7](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/assignments/assignment_yourname_class7.ipynb)